In [42]:
import Utils as utils
import Collaborative_Filtering_Utils as cf_utils

import pandas as pd
import numpy as np
import pickle

from surprise import SVD, SVDpp, SlopeOne, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import accuracy



In [ ]:
################## Load trained algorithms ##################
'''file = open('/home/Virality_Predictor/models/CF_PT_norm.pkl',"rb")
algo_norm = pickle.load(file)
file.close()

file = open('/home/Virality_Predictor/models/CF_PT_raw.pkl',"rb")
algo_raw = pickle.load(file)
file.close()'''


In [43]:
# Load articles
shared_articles_df = utils.load_shared_articles('shared_articles.csv', ['pt'])
article_indices = pd.Series(shared_articles_df.index)


/Users/mirayyuce/Desktop/Projects/Virality_Predictor/Utils.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain_name'] = df['url'].apply(split_url)


In [44]:
# Load users with raw virality values
user_interactions_df_raw = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=False)

# Loads users with normalized virality
user_interactions_df_norm = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=True)

In [45]:
print('Are there duplicate users?')
user_interactions_df_raw['personId'].nunique() != len(user_interactions_df_raw.groupby('personId'))

Are there duplicate users?


False

In [46]:
print('Number of users')
user_interactions_df_raw['personId'].nunique()

Number of users


1619

In [47]:
# Discard users with only one interaction for normalized virality
user_interactions_df_norm = utils.filter_user_interactions(user_interactions_df_norm, 1)

Number of users discarded:  351


In [48]:
# Discard users with only one interaction for raw virality
user_interactions_df_raw = utils.filter_user_interactions(user_interactions_df_raw, 1)

Number of users discarded:  351


In [49]:
# Create train and test datasets for normalized virality
norm_train_data_object, norm_test_data_object, norm_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_norm, is_normalized=True)
norm_built_full_train, norm_anti_test_set_from_train, norm_built_full_test_set, norm_train_to_test_set = cf_utils.prepare_train_test_sets(norm_train_data_object, norm_test_data_object)

normalized_sets = {'cv_train': norm_train_data_object,
                   'cv_test': norm_test_data_object,
                   'k_fold_set': norm_user_interactions_data_object,
                   'trainset': norm_built_full_train,
                   'testset' : norm_built_full_test_set,
                   'trainset_test_it' : norm_train_to_test_set,
                   'anti_trainset': norm_anti_test_set_from_train}


In [50]:
# Create train and test datasets for raw virality
raw_train_data_object, raw_test_data_object, raw_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_raw, is_normalized=False)
raw_built_full_train, raw_anti_test_set_from_train, raw_built_full_test_set, raw_train_to_test_set = cf_utils.prepare_train_test_sets(raw_train_data_object, raw_test_data_object)

raw_sets = {'cv_train': raw_train_data_object,
            'cv_test': raw_test_data_object,
            'k_fold_set': raw_user_interactions_data_object,
            'trainset': raw_built_full_train,
            'testset' : raw_built_full_test_set,
            'trainset_test_it' : raw_train_to_test_set,
            'anti_trainset': raw_anti_test_set_from_train}

In [51]:
# Search for promising algorithm using normalized data
norm_results = cf_utils.algorithm_search(normalized_sets['cv_train'])
norm_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,0.483254,0.046533,0.076529,NaN,NaN
SVDpp,0.483966,7.480192,0.470864,NaN,NaN
SVD,0.488532,0.920799,0.069843,NaN,NaN
KNNBaseline,0.510845,0.187265,0.710097,NaN,NaN
KNNWithMeans,0.511627,0.135295,0.417283,NaN,NaN
KNNWithZScore,0.513072,0.296124,0.454818,NaN,NaN
SlopeOne,0.524058,0.098673,0.171781,NaN,NaN
KNNBasic,0.533325,0.097647,1.318026,NaN,NaN
CoClustering,0.543088,1.030542,0.104669,NaN,NaN


In [52]:
# Search for promising algorithm using raw data
raw_results = cf_utils.algorithm_search(raw_sets['cv_train'])
raw_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,2.152736e+01,0.020756,0.029855,NaN,NaN
KNNBaseline,2.408879e+01,0.121031,0.324019,NaN,NaN
SlopeOne,2.486392e+01,0.076875,0.188894,NaN,NaN
KNNBasic,2.556719e+01,0.056344,0.212742,NaN,NaN
KNNWithMeans,2.567332e+01,0.115005,0.415563,NaN,NaN
CoClustering,2.591084e+01,0.757774,0.033873,NaN,NaN
KNNWithZScore,2.602815e+01,0.153762,0.214250,NaN,NaN
NormalPredictor,2.781265e+01,0.015245,0.045219,NaN,NaN
SVD,3.332901e+06,0.981935,0.073884,NaN,NaN


In [53]:
# Based on the benchmark results we pick KNNBasic as the Collaborative Filtering algorithm.
algorithm = KNNBasic

In [54]:
# Grid search parameters for KNNBasic
parameters = {'k': [40, 80, 100, 150], 
              'min_k': [5, 10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5, 10],
                              'user_based': [True, False]}
             }

In [55]:
cf_utils.grid_search(algorithm, parameters, normalized_sets['cv_train'])

<class 'surprise.model_selection.search.GridSearchCV'>
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matr

/Users/mirayyuce/.virtualenvs/thesis/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computi

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computi

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computi

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing sim

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_min_k,param_sim_options
0,0.513657,0.520258,0.510644,0.514853,0.004015,141,0.361338,0.366577,0.363246,0.363720,0.002165,13,0.080060,0.012530,0.351502,0.206972,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 1, 'user_based'..."
1,0.503098,0.504569,0.494746,0.500804,0.004326,4,0.355009,0.359335,0.353733,0.356026,0.002397,1,0.033585,0.007821,0.193949,0.013155,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 1, 'user_based'..."
2,0.514876,0.514906,0.511262,0.513681,0.001711,134,0.362986,0.366957,0.364185,0.364709,0.001663,36,0.050567,0.003935,0.200613,0.007968,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 5, 'user_based'..."
3,0.514098,0.514460,0.509914,0.512824,0.002063,33,0.362895,0.366279,0.363894,0.364356,0.001420,31,0.025209,0.000919,0.233640,0.070999,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 5, 'user_based'..."
4,0.514888,0.514643,0.510922,0.513484,0.001815,118,0.363348,0.366885,0.364685,0.364972,0.001458,125,0.047492,0.004629,0.187954,0.012781,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 10, 'user_based..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.514790,0.514587,0.510233,0.513203,0.002102,39,0.363410,0.366788,0.364333,0.364844,0.001425,41,0.310744,0.018451,0.175553,0.014868,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 1, 'user_bas..."
140,0.514939,0.514634,0.510718,0.513430,0.001922,73,0.363432,0.366870,0.364444,0.364915,0.001442,69,0.071751,0.006620,0.232440,0.007202,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 5, 'user_bas..."
141,0.514939,0.514634,0.510718,0.513430,0.001922,57,0.363432,0.366870,0.364444,0.364915,0.001442,61,0.032443,0.002210,0.180402,0.029635,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 5, 'user_bas..."
142,0.514939,0.514634,0.510718,0.513430,0.001922,101,0.363432,0.366870,0.364444,0.364915,0.001442,76,0.065444,0.005965,0.155097,0.009453,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 10, 'user_ba..."


In [56]:
cf_utils.grid_search(algorithm, parameters, raw_sets['cv_train'])

<class 'surprise.model_selection.search.GridSearchCV'>
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matr

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done comp

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...


,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_min_k,param_sim_options
0,23.260239,23.642649,23.708079,23.536989,0.197506,144,10.318116,10.383375,10.623840,10.441777,0.131466,32,0.059393,0.011120,0.207268,0.028081,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 1, 'user_based'..."
1,23.027537,22.764164,22.811997,22.867900,0.114558,133,10.479286,10.229437,10.183798,10.297507,0.129881,1,0.027007,0.002257,0.202688,0.004894,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 1, 'user_based'..."
2,22.411953,22.467141,22.687150,22.522081,0.118876,120,10.475239,10.287739,10.557098,10.440025,0.112749,28,0.053892,0.004507,0.186914,0.005755,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 5, 'user_based'..."
3,22.440361,22.356983,22.772054,22.523133,0.179275,124,10.523763,10.279649,10.659095,10.487502,0.157016,126,0.032661,0.005265,0.183198,0.015901,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 5, 'user_based'..."
4,22.326813,22.360009,22.653887,22.446903,0.146986,84,10.487847,10.298518,10.594148,10.460171,0.122267,110,0.053587,0.012140,0.170955,0.021927,"{'k': 40, 'min_k': 5, 'sim_options': {'name': ...",40,5,"{'name': 'msd', 'min_support': 10, 'user_based..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,22.343973,22.292676,22.638470,22.425039,0.152364,14,10.406129,10.213138,10.560947,10.393405,0.142277,22,0.138688,0.006408,0.201032,0.018631,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 1, 'user_bas..."
140,22.344841,22.353297,22.635777,22.444638,0.135200,38,10.493773,10.292613,10.582670,10.456352,0.121335,51,0.062478,0.006546,0.152961,0.009214,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 5, 'user_bas..."
141,22.344841,22.353297,22.635777,22.444638,0.135200,37,10.493773,10.292613,10.582670,10.456352,0.121335,50,0.029920,0.000754,0.142982,0.006216,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 5, 'user_bas..."
142,22.344841,22.353297,22.635777,22.444638,0.135200,24,10.493773,10.292613,10.582670,10.456352,0.121335,45,0.058004,0.001304,0.145974,0.011228,"{'k': 150, 'min_k': 20, 'sim_options': {'name'...",150,20,"{'name': 'cosine', 'min_support': 10, 'user_ba..."


In [33]:
# Train and test the normalized data using the best parameters

algo_norm = KNNBasic(k= 40, min_k= 5, sim_options= {'name': 'msd', 'min_support': 1, 'user_based': False})
algo_norm.fit(normalized_sets['trainset'])
predictions_norm_anti_trainset = algo_norm.test(normalized_sets['anti_trainset'])
accuracy.rmse(predictions_norm_anti_trainset)
predictions_norm_testset = algo_norm.test(normalized_sets['testset'])
accuracy.rmse(predictions_norm_testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1195
RMSE: 0.4959


0.49586877159241943

In [57]:
file = open('/home/Virality_Predictor/models/CF_PT_norm.pkl',"wb")
pickle.dump(algo_norm,file)
file.close()

In [34]:
precision5, recall5 = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision5, recall5)

Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 5 0.34345898004434605
Recall at 5 0.46356945899628826
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 5 0.34860793287566755
Recall at 5 0.4591915108791539
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 5 0.38301569506726457
Recall at 5 0.49374278176718
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 5 0.3651393661702941
Recall at 5 0.5042416154436176
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 5 0.36661067562523336
Recall at 5 0.4830624074574608
0.3613665299565611 0.4807615549087402


In [35]:
precision10, recall10 = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision10, recall10)

Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 10 0.367272865553587
Recall at 10 0.5277637717048052
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 10 0.37085225175341446
Recall at 10 0.530561978651024
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 10 0.35181020690646964
Recall at 10 0.5069833369097242
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 10 0.3624664782199029
Recall at 10 0.5136887370620248
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision at 10 0.37615577621183005
Recall at 10 0.5308768829641499
0.1828557578645204 0.2609874707291728


In [36]:
# Top 3 predictions per user

cf_utils.print_top_n_predictions_per_user(predictions_norm_testset, shared_articles_df, article_indices, 3)

User  3609194402293569455
Microsoft convoca comunidade técnica para resolver problemas usando a nuvem - Startupi on startupi.com
71 erros de português que precisam sumir dos seus e-mails on blog.runrun
A compra de alimentos online é nova fronteira do e-commerce mundial on mundodomarketing.com.br/index
  
User  548750006199694898
Ibope revela lojas de e-commerce com maior recomendação no Brasil - B!T magazine on bitmag.com
TrustedCompany anuncia novo produto no Fórum E-commerce Brasil 2016 on segs.com
Vendas da Netshoes crescem 267% em dispositivos móveis no 1º semestre - 24Brasil on 24brasil
  
User  -4133735994159009937
Nova campanha da Avon tem drag queen como estrela on brasilpost.com
  
User  6773859371279096371
Mulheres são mais criativas que homens? on meioemensagem.com
A publicidade sem ROI acaba de morrer: adapte-se ou junte-se a ela - Meio & Mensagem on meioemensagem.com
  
User  8907499588729810535
Infográfico: Algoritmos para Aprendizado de Máquina on infoq
  
User  -3542895

  
User  1202287501580555390
Parceria entre Paypal e Mastercard firmada! on pagamento
O Brasil das fintechs: conheça startups nacionais que podem fazer você sair dos bancos on startse.infomoney.com
Apple anuncia a 3ª geração da linguagem Swift para o final deste ano on blogdoiphone
  
User  6767912612638555664
A identidade visual de Budapeste para as Olimpíadas de 2024 on cutedrop.com
  
User  3581462318368446161
Startups matam. E suas vítimas são as empresas que ainda não entendem o que é a Economia do Acesso on projetodraft
  
User  674947025087085832
3 aplicações de realidade aumentada para apresentar novos produtos on mundodomarketing.com.br/index
Carros de luxo em alta, mas o alerta está aceso on epocanegocios.globo
Acesse a versão Material Design do YouTube | Google Discovery on googlediscovery
  
User  4568163094592867577
6 exemplos de como usar computação cognitiva | EXAME.com on exame.abril.com
Inovador, Nubank ganha prêmio no Vale do Silício | EXAME.com on exame.abril.com
Sua

User  -2065443854703875357
DBA BRASIL no INTEROPMIX 2016 on fabioprado
  
User  5019290970146412524
Livro: Retrospectivas Divertidas on caroli
  
User  3413008167249007087
Novo workaholic trabalha, pratica esportes e tem tempo para a família. Conheça on gq.globo
  
User  2833428826475063405
Como são escrita as risadas em japonês? - Suki Desu on skdesu
O meme 'Luiza, você está atenta?' explicou da melhor forma o machismo na área da tecnologia on hypeness.com
O que o GitHub tem a nos dizer sobre os estereótipos entre programadores | Tecnoblog on tecnoblog
  
User  6919601181993706462
Livro: Retrospectivas Divertidas on caroli
  
User  -5732707497022383920
Livro: Retrospectivas Divertidas on caroli
Psicóloga de Harvard diz que as pessoas julgam você em segundos por esses critérios | Jornal do Empreendedor on jornaldoempreendedor.com
  
User  6590718530693270591
Shopping em BH terá fazenda urbana de 2.700 m² on ciclovivo.com
  
User  -3978390417378743562
Susep divulga nota de esclareciment

User  -6014236343041475884
MOBILE TIME - FlyHelo: app permite compartilhamento de jatos e helicópteros em São Paulo on mobiletime.com
  
User  7302126548210559734
Microsoft lança Teams, nova plataforma de chat concorrente do Slack | MacMagazine.com.br on macmagazine.com
Conheça Cris Grether, a diretora Global de Design que liderou, em Atlanta, a reformulação das embalagens de Coca-Cola on cocacolabrasil.com
  
User  3773993939681666059
Novo workaholic trabalha, pratica esportes e tem tempo para a família. Conheça on gq.globo
Microsoft anuncia Visual Studio para Mac, disponível ainda nesta semana | MacMagazine.com.br on macmagazine.com
  
User  3117868596680657524
Você poderá abrir ou fechar uma conta corrente pela internet | Tecnoblog on tecnoblog
  
User  2652684983165843086
Seguro de Pessoas on cnseg.org
Johnson & Johnson comprará grupo suíço por US$ 30 bi on epocanegocios.globo
Carne artificial acabará com o agronegócio brasileiro, afirma especialista on administradores.com
  
User 

  
User  -5873251123741236991
Custo do Erro - Cinco motivos para investir em automação de testes on attps.com
  
User  4313424003136719348
Custo do Erro - Cinco motivos para investir em automação de testes on attps.com
  
User  -897315218084387621
Elektro, Google e Sama lideram entre as melhores empresas para trabalhar em 2016 on epoca.globo
  
User  1763748489948355015
Um bilhão de arquivos mostram quem vence a disputa tabs vs. espaços entre programadores on gizmodo.uol.com
  
User  7193191844620858589
Bot do Facebook Messenger simula entrevista para o processo de "3%" on b9.com
  
User  -698695976631873505
Inteligência artificial da IBM criou 'trailer perfeito' para filme on m.olhardigital.uol.com
  
User  5683958305498326032
Testes unitários vs aceitação - Android Dev BR on medium
  
User  -4246695160507476121
Arquitetura de Microserviços habilitando APIs - Sensedia on sensedia
Aplicações Financeiras Flexíveis através de DSLs na Plataforma Java on infoq
  
User  3813842765808990208


In [37]:
# Train and test the raw data using the best parameters

algo_raw = KNNBasic(k= 150, min_k= 5, sim_options= {'name': 'cosine', 'min_support': 1, 'user_based': False})
algo_raw.fit(raw_sets['trainset'])
predictions_raw_anti_trainset = algo_raw.test(raw_sets['anti_trainset'])
accuracy.rmse(predictions_raw_anti_trainset)
predictions_raw_testset = algo_raw.test(raw_sets['testset'])
accuracy.rmse(predictions_raw_testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 5.5531
RMSE: 23.0869


23.08688576098913

In [58]:
file = open('/home/Virality_Predictor/models/CF_PT_raw.pkl',"wb")
pickle.dump(algo_raw,file)
file.close()

In [38]:
precision5r, recall5r = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision5r, recall5r)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 5 0.003449568803899513
Recall at 5 0.00562429696287964
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 5 0.004118993135011442
Recall at 5 0.003384820747520976
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 5 0.002893649004133784
Recall at 5 0.003945885005636978
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 5 0.002885345482156416
Recall at 5 0.0030100878620240804
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 5 0.0048547400611620795
Recall at 5 0.0052752293577981646
0.003640459297272647 0.004248063987171968


In [39]:
precision10r, recall10r = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision10r, recall10r)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 10 0.005215419501133786
Recall at 10 0.00510204081632653
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 10 0.001887504718761797
Recall at 10 0.003963759909399773
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 10 0.004617117117117117
Recall at 10 0.0038475975975975976
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 10 0.002589715131335553
Recall at 10 0.003699593044765076
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision at 10 0.0019211388259007306
Recall at 10 0.0022675736961451248
0.001623089529424898 0.0018880565064234103


In [40]:
cf_utils.print_top_n_predictions_per_user(predictions_raw_testset, shared_articles_df, article_indices, 3)

User  3609194402293569455
Microsoft convoca comunidade técnica para resolver problemas usando a nuvem - Startupi on startupi.com
Serasa Experian realiza hackathon com prêmios de até R$10 mil - Startupi on startupi.com
O início problemático dos chatbots no Facebook Messenger on gizmodo.uol.com
  
User  548750006199694898
TrustedCompany anuncia novo produto no Fórum E-commerce Brasil 2016 on segs.com
Ibope revela lojas de e-commerce com maior recomendação no Brasil - B!T magazine on bitmag.com
Vendas da Netshoes crescem 267% em dispositivos móveis no 1º semestre - 24Brasil on 24brasil
  
User  -4133735994159009937
Nova campanha da Avon tem drag queen como estrela on brasilpost.com
  
User  6773859371279096371
Mulheres são mais criativas que homens? on meioemensagem.com
A publicidade sem ROI acaba de morrer: adapte-se ou junte-se a ela - Meio & Mensagem on meioemensagem.com
  
User  8907499588729810535
Infográfico: Algoritmos para Aprendizado de Máquina on infoq
  
User  -3542895710527099

YouTube pode lançar serviço de televisão online "Unplugged" em 2017 | Google Discovery on googlediscovery
  
User  5127372011815639401
Java 9 na prática: Jigsaw on blog.caelum.com
Natura inova e cria bot para auxiliar consumidores a escolherem presentes de natal - Startupi on startupi.com
Microsoft lança Teams, nova plataforma de chat concorrente do Slack | MacMagazine.com.br on macmagazine.com
  
User  -5868110530814399805
Dasa compra maior rede de laboratórios do Nordeste on valor.com
Uber lança serviço de helicóptero em SP com preços a partir de R$66 - IDG Now! on idgnow.com
Startups apostam em aplicativos que conectam fretes e caminhoneiros on em.com
  
User  -7456488753754080246
Gmail e Inbox by Gmail terão suporte a e-mails com design responsivo | Google Discovery on googlediscovery
Um bilhão de arquivos mostram quem vence a disputa tabs vs. espaços entre programadores on gizmodo.uol.com
Microsoft lança Teams, nova plataforma de chat concorrente do Slack | MacMagazine.com.br on m

UX ou UI? on medium
Gamification - por que somos motivados por mecânicas de jogos - Mochileiro Corporativo on exame.abril.com
  
User  8189031666388954162
MITSUBISHI ASX 2.0 4X4 AWD 16V GASOLINA 4P AUTOMÁTICO - WebMotors - 17115683 on webmotors.com
[Article] Poka Yoke: o que é e como implementar | Endeavor Brasil on endeavor.org
  
User  6666652852815687733
Java 9 na prática: Jigsaw on blog.caelum.com
  
User  8968131284214320024
Facebook vai lançar sua nova rede social corporativa no mês que vem on conteudo.startse.com
Piloto de caça criado por inteligência artificial vence humano em combate simulado on tecnologia.uol.com
Johnson & Johnson comprará grupo suíço por US$ 30 bi on epocanegocios.globo
  
User  -3535274684588209118
Bayer propõe comprar Monsanto para formar a maior agroquímica do mundo on oglobo.globo
Olhar Digital: Notícia: Testamos o HoloLens, os óculos do futuro da Microsoft on m.olhardigital.uol.com
Seja esperto no trabalho: Melhore a comunicação na empresa com 12 robôs 

Microsoft adquire LinkedIn por US$ 26,2 bilhões on computerworld.com
  
User  7527226129639571966
Quando usar paginação e quando usar scroll infinito? on arquiteturadeinformacao
Quando o Scrum irá falhar - Tableless on tableless.com
O chefe é gay. E daí? on epoca.globo
  
User  -3325508780389598615
Microsoft lança Teams, nova plataforma de chat concorrente do Slack | MacMagazine.com.br on macmagazine.com
  
User  -5150600065224095351
Psicóloga de Harvard diz que as pessoas julgam você em segundos por esses critérios | Jornal do Empreendedor on jornaldoempreendedor.com
  
User  -5281916878353077993
O manual do deboísta corporativo on linkedin
  
User  -5157376071499052964
Domain Driven Design e Microservices on infoq
Docker e .Net on mundodocker.com
  
User  6860472782621321785
Ciclistas relatam roubos com agressividade na avenida dos Andradas on otempo.com
A minha viagem à Maternidade #tetodomundo on viagensdealline
  
User  -2922237472410262877
Sabia que você pode transformar o seu ce

Ciesp-Campinas e Lean Institute Brasil fazem evento gratuito sobre "lean para superar a crise" em Campinas on .portalnovidade.com
Carne artificial acabará com o agronegócio brasileiro, afirma especialista on administradores.com
  
User  -4028919343899978105
Ser um bom líder depende de inúmeros fatores. Mas um deles, segundo o inglês Simon Sinek, é importantíssimo: a coragem para proteger as equipes e para arriscar quando necessário on vocesa.uol.com
Google lança YouTube Kids no Brasil | Google Discovery on googlediscovery
Sons de animais na busca do Google | Google Discovery on googlediscovery
  
User  -8674958742744576254
Google lança novos recursos para o Analytics on adnews.com
Microsoft lança Teams, nova plataforma de chat concorrente do Slack | MacMagazine.com.br on macmagazine.com
Johnson & Johnson comprará grupo suíço por US$ 30 bi on epocanegocios.globo
  
User  -331066625167168067
Google Duo, aplicativo de chamadas em vídeo | Google Discovery on googlediscovery
Curta do Google

Bancos brasileiros fecham 200 agências em 2015 - Home - iG on economia.ig.com
Por que o segundo maior banco da Alemanha decidiu entrar no Brasil com R$ 240 milhões? on infomoney.com
UX ou UI? on medium
  
User  212272079522596410
Comercial japonês cria cidade de papel com caneta que conduz eletricidade on b9.com
  
User  -6650942110522720391
Investimentos em coaching e mentoring aumentam nas empresas on vocerh.uol.com
  
User  1949009070102523745
Seja esperto no trabalho: Melhore a comunicação na empresa com 12 robôs on br.blog.trello
Um bilhão de arquivos mostram quem vence a disputa tabs vs. espaços entre programadores on gizmodo.uol.com
Novo padrão de rede cabeada pode oferecer até 5 Gbps transferência de dados on techtudo.com
  
User  -8763398617720485024
Referências para Estudo - Xamarin on xamarinbr.azurewebsites
A importância dos filmes de mulherzinha - Capitolina on revistacapitolina.com
71 erros de português que precisam sumir dos seus e-mails on blog.runrun
  
User  -65333046

User  3014865794683484443
O futuro das interfaces: Chatbots - Tableless on tableless.com
  
User  7660434827542910956
Robô acha de almoço de 12 kg a cervejas pagas pelo cidadão on otempo.com
  
User  -492179582639058400
Nokia vai voltar ao mercado de smartphones on ztop.com
Carrinho de criança carrega seu smartphone usando geradores instalados nas rodas on m.gizmodo.uol.com
  
User  -8123627990288459252
Ganhe 6 meses de acesso ao Pluralsight, maior plataforma de treinamento online on medium
  
User  2032181124951647555
Gamification - por que somos motivados por mecânicas de jogos - Mochileiro Corporativo on exame.abril.com
  
User  1716230101302949859
Dicionário de Segurês on segfoco.com
  
User  -4139451785876182601
Bayer anuncia compra da Monsanto por US$ 66 bilhões on g1.globo
  
User  4156846511612859158
Google vai reduzir em 50% consumo de memória do Chrome | Google Discovery on googlediscovery
  
User  -4744441833918341987
Carne artificial acabará com o agronegócio brasileiro, af

User  -6120788038252469111
MasterTech - Como ter um bom relacionamento com a Escrita on mastertech
  
User  8214844442027962164
Cade autoriza Itaú Unibanco e Mastercard a criarem nova bandeira de cartão on infomoney.com
  
User  -8606085472606356565
Ser um bom líder depende de inúmeros fatores. Mas um deles, segundo o inglês Simon Sinek, é importantíssimo: a coragem para proteger as equipes e para arriscar quando necessário on vocesa.uol.com
  
User  8418615305660062636
Livro: Retrospectivas Divertidas on caroli
  
User  -3458422503840091409
Java Garbage Collection Essencial on infoq
1º Congresso Brasileiro de IoT - Embarcados on embarcados.com
Quando o Scrum irá falhar - Tableless on tableless.com
  
User  -4111723879505360159
Arquitetura de Microserviços habilitando APIs - Sensedia on sensedia
  
User  8879844298911979276
ProtoHack, hackathon criado no Vale do Silício, desembarca em São Paulo para sua primeira edição no espaço Engenho Maker - Startupi on startupi.com
Ganhe 6 meses de

User  8437083040455072898
Conheça Cris Grether, a diretora Global de Design que liderou, em Atlanta, a reformulação das embalagens de Coca-Cola on cocacolabrasil.com
  
User  -2041685972080499493
Itaú é pioneiro do blockchain on baguete.com
  
User  -6606373800483513352
É chegada a hora do desenvolvimento de software orientado por hipóteses - CIO on cio.com
  
User  5489180667547554980
Psicóloga de Harvard diz que as pessoas julgam você em segundos por esses critérios | Jornal do Empreendedor on jornaldoempreendedor.com
  
User  -2186033964866370752
O manual do deboísta corporativo on linkedin
  
User  -6003416708481128042
Johnson & Johnson comprará grupo suíço por US$ 30 bi on epocanegocios.globo
  
User  -8384598282283588647
Por que a limitação da banda larga é uma forma de ignorar o futuro on medium
  
User  -6411371239765052331
Livro: Retrospectivas Divertidas on caroli
  
User  -9188188261933657343
Lava-louças Brastemp Ative! 8 Serviços Blf08abana 110v - R$ 800,00 on produto.merca